# Electricity Markets

This example gradually builds up more and more complicated energy-only electricity markets in PyPSA, starting from a single bidding zone, going up to multiple bidding zones connected with transmission (NTCs) along with variable renewables and storage.

## Preliminaries

In [41]:
import numpy as np
import pypsa

In [42]:
# marginal costs in EUR/MWh
marginal_costs = {"Wind": 0, "Hydro": 0, "Coal": 30, "Gas": 60, "Oil": 80}

# power plant capacities (nominal powers in MW) in each country (not necessarily realistic)
power_plant_p_nom = {
    "South Africa": {"Coal": 35000, "Wind": 3000, "Gas": 8000, "Oil": 2000},
    "Mozambique": {"Hydro": 1200},
    "Eswatini": {"Hydro": 600},
}

# transmission capacities in MW (not necessarily realistic)
transmission = {
    "South Africa": {"Mozambique": 500, "Eswatini": 250},
    "Mozambique": {"Eswatini": 100},
}

# country electrical loads in MW (not necessarily realistic)
loads = {"South Africa": 42000, "Mozambique": 650, "Eswatini": 250}

## Single bidding zone with fixed load, one period

In this example we consider a single market bidding zone, South Africa.

The inelastic load has essentially infinite marginal utility (or higher than the marginal cost of any generator).

#### Mathematical formulation of the problem (linear economic dispatch problem)

The problem has
* One bidding zone / one bus: South Africa 
* One time period
* Fixed (inelastic) load
* Several generators with:
    * Capacity limits
    * Constant marginal costs
* No transmission, no storage, no ramping, no unit commitment

Decision variables:
4 decision variables for the power output of Coal, Wind, Gas, Oil generators *g*. 

Parameters:
1. Generator capacities: $0 \leq p_g \leq \bar{P}_g$

**Generator**  | **$\bar{P}_g$ (MW)**  | **Marginal cost $c_g$ (Euro/MWh)**

Coal                | 35,000            | 30

Wind                | 3,000             | 0

Gas                 | 8,000             | 60

Oil                 | 2,000             | 80

2. Load (fixed demand): $D = 42,000$ MW

Objective function:

PyPSA minimizes total generation cost:
$$\min_{p_g} = \sum_g c_g p_g$$
Explicitly:
$$\min  0\cdot p_{Wind} + 30\cdot p_{Coal} + 60\cdot p_{Gas} + 80\cdot p_{Oil}$$

This is a linear objective, which can be solved as an LP

Constraints:

(a) Power Balance (energy conservation): Because load is inelastic and must be met:
$$\sum_g p_g = D$$
$$p_{Wind} + p_{Coal} + p_{Gas} + p_{Oil} = 42,000$$

This is the most important constraint. is has associated dual variable, which becomes the market clearing price.

(b) Generator capcity limits: For each generator $0 \leq p_g \leq \bar{P}_g$. This is a simply box constraint.

How the optimizer reasons: (merit order principle)

Because costs are linear and there are no other cosntraints:
1. Dispatch cheapest generators first
2. Fill each generator up to its capacity
3. Stop when demand is met

The dual variable of the power balance constraint ($\lambda$) is the marginal cost of supplying one additional MW of demand. Generators deployed before the last one earn marginal price, the last generator breaks even. Costlier plants are not dispatched. 

In [43]:
country = "South Africa"

n = pypsa.Network()

n.add("Bus", country)

for tech in power_plant_p_nom[country]:
    n.add(
        "Generator",
        f"{country} {tech}",
        bus=country,
        p_nom=power_plant_p_nom[country][tech],
        marginal_cost=marginal_costs[tech],
    )


n.add("Load", f"{country} load", bus=country, p_set=loads[country]);

In [44]:
n.optimize()

Index(['South Africa'], dtype='object', name='name')
INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io: Writing time: 0.02s
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 4 primals, 9 duals
Objective: 1.29e+06
Solver model: available
Solver message: Optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper were not assigned to the network.


Running HiGHS 1.12.0 (git hash: 755a8e0): Copyright (c) 2025 HiGHS under MIT licence terms
LP linopy-problem-yfrl7jkp has 9 rows; 4 cols; 12 nonzeros
Coefficient ranges:
  Matrix  [1e+00, 1e+00]
  Cost    [3e+01, 8e+01]
  Bound   [0e+00, 0e+00]
  RHS     [2e+03, 4e+04]
Presolving model
1 rows, 3 cols, 3 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve reductions: rows 0(-9); columns 0(-4); nonzeros 0(-12) - Reduced to empty
Performed postsolve
Solving the original LP from the solution after postsolve

Model name          : linopy-problem-yfrl7jkp
Model status        : Optimal
Objective value     :  1.2900000000e+06
P-D objective error :  0.0000000000e+00
HiGHS run time      :          0.00


('ok', 'optimal')

Print the load active power (P) consumption:

In [45]:
n.loads_t.p

name,South Africa load
snapshot,
now,42000.0


Print the generator active power (P) dispatch:

In [46]:
n.generators_t.p

name,South Africa Coal,South Africa Wind,South Africa Gas,South Africa Oil
snapshot,,,,
now,35000.0,3000.0,4000.0,-0.0


Print the clearing price, which corresponds to gas:

In [47]:
n.buses_t.marginal_price

name,South Africa
snapshot,
now,60.0


## Two bidding zones connected by transmission, one period

In this example we have bidirectional lossless transmission capacity between two bidding zones. The power transfer is treated as controllable (like an A/NTC (Available/Net Transfer Capacity) or HVDC line). Note that in the physical grid, power would flow passively according to the network impedances.

This problem has:
1. Two buses (bidding zones)
2. Generators attached to each bus
3. Loads at each bus
4. A controllable, lossless transmission link with capacity limits

This is a network-constrained economic dispatch problem.

Decision variables:

(a) Generator dispatch: For each generator *g* at bus *i*:
$$p_{g,i} \geq 0$$

(b) Transmission flow: For each link *l* from South Africa -> Mozambique
$$f_l \in [-\bar{F}_l, \bar{F}_l]$$
* Negative flow means power flows from Mozambique to SA
* This matches p_min_pu = -1

There are 6 primal variables

Parameters:

(a) Loads: $D_{SA} = 42,000$ and $D_{MX} = 650$

(b) Generator capacities and costs: Same as above, but split by country

(c) Transmission capacity: $\bar{F} =500$ MW. Lossless and controllable.

Objective function:

The system operator minimizes total generation cost across both zones
$$\min \sum_{i\in {SA,MZ}} \sum_{g\in i} c_g p_{g,i}$$

There is no cost on transmission.

Power balance constraints (key difference from before):

Each bidding zone has its own nodal balance.

* Mozambique balance:
$$\sum_{g\in MZ} p_{g,MZ} - f = D_{MZ}$$
 * If $f<0$, Mozambique exports power
 * If $f>0$, Mozambique imports power
* SA balance:
$$\sum_{g\in SA} p_{g,SA} + f = D_{SA}$$

Generator constraints:

(same as before)

Transmission constraint:
$$-\bar{F} \leq f \leq \bar{F}$$

Merit order with trade:
* Local marginal costs
    * Mozambique: Hydro =0 
    * SA: cheapest marginal unit after wind+coal = Gas = 60
* So Mozambique has cheaper marginal generation than SA. It has negative flow of 500 MW to SA, which is the transmission limit. The line is congested.

Prices (dual variables of nodal balances):
1. Mozambique price: $\lambda_{MZ} = 0$
2. SA price: $\lambda_{SA} = 60$
3. Interpretation: Because the link is congested, $\lambda_{SA} \neq \lambda_{MZ}$. If transmission were infinite, price would equalize at 0 Euro/MWh.

** Total system cost dropped because 500 MW of gas was replaced by hydro - 500*60 = 30,000 Euros saved.

Transmission shadow price (congestion rent):

The dual variable on the transmission capacity constraint is:
$$\mu = \lambda_{SA} - \lambda_{MZ} = 60$$
Economic meaning:
* Value of 1 more MW of transmission capacity
* Also called congestion rent per MW
Total congestion rent = 30,000

Exactly equal to the welfare gain from trade.

This is the mathematical core of:
1. Market coupling
2. Locational marginal pricing
3. Cross-border electricity trade

In [48]:
n = pypsa.Network()

countries = ["Mozambique", "South Africa"]

for country in countries:
    n.add("Bus", country)

    for tech in power_plant_p_nom[country]:
        n.add(
            "Generator",
            f"{country} {tech}",
            bus = country,
            p_nom = power_plant_p_nom[country][tech],
            marginal_cost = marginal_costs[tech],
        )
    n.add("Load", f"{country} load", bus=country, p_set = loads[country])

    # add transmission as controllable link
    if country not in transmission:
        continue
    
    for other_country in countries:
        if other_country not in transmission[country]:
            continue

        #Note: Link is default unidirectional, so have to set p_min_pu = -1
        # to allow bidirectional (also negative) flow

        n.add(
            "Link",
            f"{country} - {other_country} link",
            bus0 = country,
            bus1 = other_country,
            p_nom = transmission[country][other_country],
            p_min_pu = -1,
        )

In [49]:
n.optimize()

Index(['Mozambique', 'South Africa'], dtype='object', name='name')
Index(['South Africa - Mozambique link'], dtype='object', name='name')
INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io: Writing time: 0.03s
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 6 primals, 14 duals
Objective: 1.26e+06
Solver model: available
Solver message: Optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Link-fix-p-lower, Link-fix-p-upper were not assigned to the network.


Running HiGHS 1.12.0 (git hash: 755a8e0): Copyright (c) 2025 HiGHS under MIT licence terms
LP linopy-problem-cvsrs5jc has 14 rows; 6 cols; 19 nonzeros
Coefficient ranges:
  Matrix  [1e+00, 1e+00]
  Cost    [3e+01, 8e+01]
  Bound   [0e+00, 0e+00]
  RHS     [5e+02, 4e+04]
Presolving model
1 rows, 3 cols, 3 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve reductions: rows 0(-14); columns 0(-6); nonzeros 0(-19) - Reduced to empty
Performed postsolve
Solving the original LP from the solution after postsolve

Model name          : linopy-problem-cvsrs5jc
Model status        : Optimal
Objective value     :  1.2600000000e+06
P-D objective error :  0.0000000000e+00
HiGHS run time      :          0.00


('ok', 'optimal')

In [50]:
n.loads_t.p

name,Mozambique load,South Africa load
snapshot,,
now,650.0,42000.0


In [51]:
n.generators_t.p

name,Mozambique Hydro,South Africa Coal,South Africa Wind,South Africa Gas,South Africa Oil
snapshot,,,,,
now,1150.0,35000.0,3000.0,3500.0,-0.0


In [52]:
n.buses_t.marginal_price

name,Mozambique,South Africa
snapshot,,
now,-0.0,60.0


In [53]:
n.links_t.p0

name,South Africa - Mozambique link
snapshot,
now,-500.0


The shadow prices of the links measure the inframarginal rent of the link, i.e. the difference between the marginal price of the two bidding zones.

In [54]:
n.links_t.mu_lower

name
snapshot
now


## Three bidding zones connected by transmission, one period

In this example we have bidirectional lossless transmission capacity between three bidding zones. The power transfer is treated as controllable (like an A/NTC (Available/Net Transfer Capacity) or HVDC line). Note that in the physical grid, power would flow passively according to the network impedances. The links form a meshed but capacity-constrained trading network, not a physical AC grid.

**Decision variables:**

(a) Generator dispatch: For each generator *g* at bus *i*:
$$p_{g,i} \geq 0$$

(b) Transmission flows: For each directed link $l = (i\rightarrow j)$
$$f_{ij} \in [-\bar{F}_{ij}, \bar{F}_{ij}]$$
Negative flow means power flows from *j* to *i*. 

** 6 generator variables + 3 link flow variables = 9 primals

**Parameters:**

(a) Loads: each country has it's own inelastic demand

(b) Generator capacity and marginal costs: Each country has different generation sources with their respective marignal costs and capacities.

(c) Transmission capacity: There is a limit to how much power can be tradded between each country. 

**Objective function:**

Minimizes total system generation costs, and we assume transmission is free and lossless.
$$\min \sum_{i} \sum_{g\in i}  c_g p_{g,i}$$

**Nodal power balance constraints:**

Each bidding zone must satisfy its own energy balance.

(a) Eswatini: $p_{E, hydro} + f_{M,E} + f_{SA, E} + D_E$

(b) Mozambique: $p_{M, hydro} - f_{M,E} + f_{SA, M} + D_M$

(c) South Africa: $\sum_{g\in SA} p_{g, SA} - f_{SA,E} - f_{SA, M} + D_{SA}$

Each equation has a dual variable, which are the zonal clearing prices: $\lambda_E, \lambda_M, \lambda_{SA}$

**Other contraints:**

(a) Generator limits: $0 \leq p_{g,i} \leq \bar{P}_{g,i}$

(b) Transmission limits: $-\bar{F}_{ij} \leq f_{ij} \leq \bar{F}_{ij}$

**Observations:**

* Total cost keeps falling as we add hydro to SA's power mix and the gas consumption decreases.
* *Congestion rent*: For any congested link $i <-> j, \mu_{ij} = \lambda_j -\lambda_i$. Since $\lambda_{SA} = 60$ and $\lambda_E = \lambda_M = 0$, each MW of extra capacity into South Africa is worth 60 €/MWh.

In [55]:
n = pypsa.Network()

countries = ["Eswatini", "Mozambique", "South Africa"]

for country in countries:
    n.add("Bus", country)

    for tech in power_plant_p_nom[country]:
        n.add(
            "Generator",
            f"{country} {tech}",
            bus=country,
            p_nom=power_plant_p_nom[country][tech],
            marginal_cost=marginal_costs[tech],
        )

    n.add("Load", f"{country} load", bus=country, p_set=loads[country])

    if country not in transmission:
        continue

    for other_country in countries:
        if other_country not in transmission[country]:
            continue

        n.add(
            "Link",
            f"{country} - {other_country} link",
            bus0=country,
            bus1=other_country,
            p_nom=transmission[country][other_country],
            p_min_pu=-1,
        )

In [56]:
n.optimize()

Index(['Eswatini', 'Mozambique', 'South Africa'], dtype='object', name='name')
Index(['Mozambique - Eswatini link', 'South Africa - Eswatini link',
       'South Africa - Mozambique link'],
      dtype='object', name='name')
INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io: Writing time: 0.04s
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 9 primals, 21 duals
Objective: 1.24e+06
Solver model: available
Solver message: Optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Link-fix-p-lower, Link-fix-p-upper were not assigned to the network.


Running HiGHS 1.12.0 (git hash: 755a8e0): Copyright (c) 2025 HiGHS under MIT licence terms
LP linopy-problem-8mqa0q7v has 21 rows; 9 cols; 30 nonzeros
Coefficient ranges:
  Matrix  [1e+00, 1e+00]
  Cost    [3e+01, 8e+01]
  Bound   [0e+00, 0e+00]
  RHS     [1e+02, 4e+04]
Presolving model
2 rows, 5 cols, 6 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve reductions: rows 0(-21); columns 0(-9); nonzeros 0(-30) - Reduced to empty
Performed postsolve
Solving the original LP from the solution after postsolve

Model name          : linopy-problem-8mqa0q7v
Model status        : Optimal
Objective value     :  1.2450000000e+06
P-D objective error :  0.0000000000e+00
HiGHS run time      :          0.00


('ok', 'optimal')

In [57]:
n.loads_t.p

name,Eswatini load,Mozambique load,South Africa load
snapshot,,,
now,250.0,650.0,42000.0


In [58]:
n.generators_t.p

name,Eswatini Hydro,Mozambique Hydro,South Africa Coal,South Africa Wind,South Africa Gas,South Africa Oil
snapshot,,,,,,
now,600.0,1050.0,35000.0,3000.0,3250.0,-0.0


In [59]:
n.buses_t.marginal_price

name,Eswatini,Mozambique,South Africa
snapshot,,,
now,-0.0,-0.0,60.0


In [60]:
n.links_t.p0

name,Mozambique - Eswatini link,South Africa - Eswatini link,South Africa - Mozambique link
snapshot,,,
now,-100.0,-250.0,-500.0


In [61]:
n.links_t.mu_lower

name
snapshot
now


## Single bidding zone with price-sensitive industrial load, one period

In this example we consider a single market bidding zone, South Africa. Now there is a large industrial load with a marginal utility which is low enough to interact with the generation marginal cost. See also the demand elasticity example.

Switche from pure cost minimization to welfare maximization with elastic demand. PyPSA implements this in a very elegant way by modeling price-sensitive demand as a negative generator.

**Changes compared to single-zone case**
* Previously the system had to serve 42,000 MW no matter the price. Now there is a large industrial consumer that is willing to consume up to 8,000 MW, only if the price is ≤ 70 €/MWh

**Key modeling trick: “negative generator”**

The variable $p_I \in [-8000, 0]$. $p<0 \rightarrow$ is consumption, $-p_I$ is industrial consumption. 

**Decision variables:**
* $p_g \geq 0 = $ output of generation unit $g$
* $p_I \leq 0 =$ indsutrial demand (negative generation)

**Objective function:**
Minimizes total cost, but now costs include negative consumption $$\min \left( \sum_g c_g p_g +70\cdot p_I \right )$$
Since $p_I \leq 0$, this term is $$70\cdot p_I = -70 \cdot (-p_I)$$
So the model is equivalently, maximising utility minus generation cost $$\max \left( 70\cdot (-p_I) - \sum_g c_g p_g \right)$$

**Constraints:**
(a) Power Balance: Let $D_0 = 42,000$ be the fixed load $$\sum_g p_g + p_I = D_0 \quad or \quad equivalently \quad \sum_g p_g = D_0 + (-p_I)$$

(b) Generator limits: $$ 0 \leq p_g \leq \bar{P}_g$$

(c) Industrial Demand limit: $$-8000 \leq p_I \leq 0$$

**Market clearing logic:**

The model will serve industrial load only if market price $\leq 70$. So industrial load is supplied by gas, not oil.

In [62]:
country = "South Africa"

n = pypsa.Network()

n.add("Bus", country)

for tech in power_plant_p_nom[country]:
    n.add(
        "Generator",
        f"{country} {tech}",
        bus=country,
        p_nom=power_plant_p_nom[country][tech],
        marginal_cost=marginal_costs[tech],
    )

# standard high marginal utility consumers
n.add("Load", f"{country} load", bus=country, p_set=loads[country])

# add an industrial load as a negative-dispatch generator with marginal utility of 70 EUR/MWh for 8000 MW
n.add(
    "Generator",
    f"{country} industrial load",
    bus=country,
    p_max_pu=0,
    p_min_pu=-1,
    p_nom=8000,
    marginal_cost=70,
);

In [63]:
n.optimize()

Index(['South Africa'], dtype='object', name='name')


INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io: Writing time: 0.02s
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 5 primals, 11 duals
Objective: 1.25e+06
Solver model: available
Solver message: Optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper were not assigned to the network.


Running HiGHS 1.12.0 (git hash: 755a8e0): Copyright (c) 2025 HiGHS under MIT licence terms
LP linopy-problem-khaj88hp has 11 rows; 5 cols; 15 nonzeros
Coefficient ranges:
  Matrix  [1e+00, 1e+00]
  Cost    [3e+01, 8e+01]
  Bound   [0e+00, 0e+00]
  RHS     [2e+03, 4e+04]
Presolving model
1 rows, 4 cols, 4 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve reductions: rows 0(-11); columns 0(-5); nonzeros 0(-15) - Reduced to empty
Performed postsolve
Solving the original LP from the solution after postsolve

Model name          : linopy-problem-khaj88hp
Model status        : Optimal
Objective value     :  1.2500000000e+06
P-D objective error :  0.0000000000e+00
HiGHS run time      :          0.00


('ok', 'optimal')

In [64]:
n.loads_t.p

name,South Africa load
snapshot,
now,42000.0


A look at the generator dispatch shows that only half of industrial load is served, because this maxes out gas; oil is too expensive with a marginal cost of 80 EUR/MWh compared to the industrial load marginal utility of 70 EUR/MWh.

In [65]:
n.generators_t.p

name,South Africa Coal,South Africa Wind,South Africa Gas,South Africa Oil,South Africa industrial load
snapshot,,,,,
now,35000.0,3000.0,8000.0,-0.0,-4000.0


In [66]:
n.buses_t.marginal_price

name,South Africa
snapshot,
now,70.0


## Single bidding zone with fixed load, several periods

In this example we consider a single market bidding zone, South Africa. We consider multiple time periods (labelled [0,1,2,3]) to represent variable wind generation.

** Multi-period (time-indexed) linear program, but still without intertemporal coupling (no storage, ramping, unit commitment). Each time period is economically independent (except they share the same installed capacities).

**Time indexing**
in PyPSA every variable and constraint is now defined per snapshot.

In [36]:
country = "South Africa"

n = pypsa.Network()

n.set_snapshots(range(4))

n.add("Bus", country)

# availability (p_max_pu) is variable for wind
for tech in power_plant_p_nom[country]:
    n.add(
        "Generator",
        f"{country} {tech}",
        bus=country,
        p_nom=power_plant_p_nom[country][tech],
        marginal_cost=marginal_costs[tech],
        p_max_pu=([0.3, 0.6, 0.4, 0.5] if tech == "Wind" else 1),
    )

# load which varies over the snapshots
n.add(
    "Load",
    f"{country} load",
    bus=country,
    p_set=loads[country] + np.array([0, 1000, 3000, 4000]),
);

In [37]:
n.optimize()

Index(['South Africa'], dtype='object', name='name')
INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io: Writing time: 0.02s
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 16 primals, 36 duals
Objective: 6.08e+06
Solver model: available
Solver message: Optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper were not assigned to the network.


Running HiGHS 1.12.0 (git hash: 755a8e0): Copyright (c) 2025 HiGHS under MIT licence terms
LP linopy-problem-on1v9_1m has 36 rows; 16 cols; 48 nonzeros
Coefficient ranges:
  Matrix  [1e+00, 1e+00]
  Cost    [3e+01, 8e+01]
  Bound   [0e+00, 0e+00]
  RHS     [9e+02, 5e+04]
Presolving model
4 rows, 12 cols, 12 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve reductions: rows 0(-36); columns 0(-16); nonzeros 0(-48) - Reduced to empty
Performed postsolve
Solving the original LP from the solution after postsolve

Model name          : linopy-problem-on1v9_1m
Model status        : Optimal
Objective value     :  6.0820000000e+06
P-D objective error :  0.0000000000e+00
HiGHS run time      :          0.00


('ok', 'optimal')

In [38]:
n.loads_t.p

name,South Africa load
snapshot,
0,42000.0
1,43000.0
2,45000.0
3,46000.0


In [39]:
n.generators_t.p

name,South Africa Coal,South Africa Wind,South Africa Gas,South Africa Oil
snapshot,,,,
0,35000.0,900.0,6100.0,-0.0
1,35000.0,1800.0,6200.0,-0.0
2,35000.0,1200.0,8000.0,800.0
3,35000.0,1500.0,8000.0,1500.0


In [40]:
n.buses_t.marginal_price

name,South Africa
snapshot,
0,60.0
1,60.0
2,80.0
3,80.0


## Single bidding zone with fixed load and storage, several periods

In this example we consider a single market bidding zone, South Africa. We consider multiple time periods (labelled [0,1,2,3]) to represent variable wind generation. Storage is allowed to do price arbitrage to reduce oil consumption.